In [187]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor
import openpyxl 
from catnip.fla_sharepoint import FLA_Sharepoint

In [188]:
nightly = pd.read_excel("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\Data Science\Resources\\2023-24 Temporary Report Data\\23-24 Panthers Nightly Data TSS Temp.xlsx")
plans = pd.read_excel("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\Data Science\Resources\\2023-24 Temporary Report Data\\23-24 Panthers Subscription Data TSS Temp.xlsx")

In [189]:
df = nightly.groupby(by = ['Plan Type'])[['Gross Revenue','Net Revenue','Tickets']].sum()
df.reset_index(inplace = True)
df = df[df['Plan Type'].apply(lambda x: len(str(x))>1)]
df['FSE'] = 0
df1 = nightly.groupby(by = ['Plan Type'])['Customer ID'].nunique()
df1 = df1.to_frame()
df1.reset_index(inplace = True)
df = df.merge(right = df1, how = 'left', on = ['Plan Type'])
df['Gross Paid ATP'] = [x/y for x,y in zip(df['Gross Revenue'],df['Tickets'])]
df['Net Paid ATP'] = [x/y for x,y in zip(df['Net Revenue'],df['Tickets'])]
df['Ticket Type'] = 0
df['Recency'] = 0
df = df[['Plan Type', 'Ticket Type','Recency','Gross Revenue', 'Net Revenue', 'FSE', 'Tickets','Customer ID', 'Gross Paid ATP',
          'Net Paid ATP']].rename(columns = {'Tickets' : 'Paid Seats', 'Customer ID' : 'Unique Accounts'})

In [190]:
df2 = plans.groupby(by = ['Plan Type', 'PC2', 'Ticket Type', 'Recency'])[['Gross Revenue','Net Revenue','Tickets']].sum()
df2.reset_index(inplace = True)
df2 = df2[df2['Plan Type'].apply(lambda x: len(str(x))>1)]
df2['Paid Seats'] = [x*y for x,y in zip([42, 42, 42, 42, 42, 42, 42, 42, 25, 25, 25, 25, 42, 42, 42, 25, 25, 42, 42, 42], df2['Tickets'])]
df2 = df2.groupby(by = ['Plan Type', 'Ticket Type', 'Recency'])[['Gross Revenue','Net Revenue','Tickets', 'Paid Seats']].sum()
df2.reset_index(inplace = True)
df3 = plans.groupby(by = ['Plan Type', 'Ticket Type', 'Recency'])['Customer ID'].nunique()
df3 = df3.to_frame()
df3.reset_index(inplace = True)
df2 = df2.merge(right = df3, how = 'left', on = ['Plan Type', 'Ticket Type', 'Recency'])
df2['Gross Paid ATP'] = [x/y for x,y in zip(df2['Gross Revenue'],df2['Paid Seats'])]
df2['Net Paid ATP'] = [x/y for x,y in zip(df2['Net Revenue'],df2['Paid Seats'])]
df2 = df2.rename(columns = {'Tickets' : 'FSE', 'Customer ID' : 'Unique Accounts'})
df2

,Plan Type,Ticket Type,Recency,Gross Revenue,Net Revenue,FSE,Paid Seats,Unique Accounts,Gross Paid ATP,Net Paid ATP
0,Annual Suites,,New Business,1333375.40,1.246145e+06,183,7686,21,173.481056,162.131822
1,Annual Suites,,Renewal,4199160.20,3.924449e+06,871,36582,45,114.787606,107.278148
2,Full Season,General,New Business,3966362.40,3.706881e+06,1636,68712,618,57.724450,53.948084
3,Full Season,General,Renewal,9282245.70,8.674996e+06,4278,179676,1635,51.661022,48.281329
4,Full Season,General,Upgrade,323388.24,3.022320e+05,143,6006,64,53.844196,50.321678
5,Full Season,Premium,New Business,1380377.04,1.120182e+06,158,6636,62,208.013418,168.803797
6,Full Season,Premium,Renewal,7200813.40,5.834082e+06,905,38010,321,189.445235,153.488086
7,Full Season,Premium,Upgrade,177530.12,1.432360e+05,30,1260,14,140.896921,113.679365
8,Premier Plan,General,New Business,1723930.50,1.611150e+06,844,21100,386,81.702867,76.357820
9,Premier Plan,General,Renewal,3130846.75,2.926025e+06,1787,44675,828,70.080509,65.495803


In [191]:
logitix = pd.read_excel("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\logitix_2324.xlsx")
logitix

,Plan Type,Ticket Type,Recency,Gross Revenue,Net Revenue,FSE,Paid Seats,Unique Accounts,Gross Paid ATP,Net Paid ATP
0,Secondary,0,0,4731333.33,4421806.85,2500,111000,6,45,42


In [192]:
dffinal = pd.concat([df,df2,logitix])
dffinal = dffinal[dffinal['Plan Type'].apply(lambda x: len(str(x))>1)]
dffinal = dffinal.reset_index()
goal_list = [1000000,3250000,2250000,4950000,5250000,5250000,21700000,21700000,21700000,21700000,21700000,21700000,
             7000000,7000000,7000000,7000000,1350000,1350000,1350000,1350000,700000,700000,700000,5050000]
dffinal['bugdet_goal'] = goal_list
dffinal

,index,Plan Type,Ticket Type,Recency,Gross Revenue,Net Revenue,FSE,Paid Seats,Unique Accounts,Gross Paid ATP,Net Paid ATP,bugdet_goal
0,0,Flex,0,0,280161.01,2.622920e+05,0,2962,188,94.585081,88.552330,1000000
1,1,Group,0,0,544844.85,4.728480e+05,0,7120,141,76.523153,66.411236,3250000
2,2,Nightly Suites,0,0,811271.00,7.864103e+05,0,3797,114,213.661048,207.113590,2250000
3,3,Singles,0,0,259113.40,2.518430e+05,0,2797,810,92.639757,90.040400,4950000
4,0,Annual Suites,,New Business,1333375.40,1.246145e+06,183,7686,21,173.481056,162.131822,5250000
5,1,Annual Suites,,Renewal,4199160.20,3.924449e+06,871,36582,45,114.787606,107.278148,5250000
6,2,Full Season,General,New Business,3966362.40,3.706881e+06,1636,68712,618,57.724450,53.948084,21700000
7,3,Full Season,General,Renewal,9282245.70,8.674996e+06,4278,179676,1635,51.661022,48.281329,21700000
8,4,Full Season,General,Upgrade,323388.24,3.022320e+05,143,6006,64,53.844196,50.321678,21700000
9,5,Full Season,Premium,New Business,1380377.04,1.120182e+06,158,6636,62,208.013418,168.803797,21700000


In [193]:
def get_pi_credentials():

    credentials = {
        "username": "panthersinsights@floridapanthers.com",
        "password": Secret.load("panthersinsights-email-password").get()
    }

    return credentials

def load(pi_creds, df: pd.DataFrame) -> None:

    FLA_Sharepoint(**pi_creds).upload_dataframe(
        df = dffinal,
        folder_path = "Resources/2023-24 Temporary Report Data",
        file_name = "union_2324",
        as_csv = True,
        add_log_date = False
    )

    return None 

with ThreadPoolExecutor(1) as pool:
    pi_creds = pool.submit(lambda: get_pi_credentials()).result()